<title>Data understanding</title>

<h1>Data understanding</h1>

In [27]:
import pyspark
from pyspark.sql.functions import sum
from pyspark.sql.functions import when, col

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('missing').getOrCreate()

<h2>Import datasets</h2>

In [5]:
app_raw = spark.read.csv('application_data.csv', header=True, inferSchema=True)
prev_raw = spark.read.csv('bureau.csv', header=True, inferSchema=True)

In [4]:
app_raw.show()

+----------+------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+--------------------------+----------+-------------+-----------------+---------------+-----------+----------+--------------+---------------+----------------+----------+----------+---------------+---------------+--------------------+---------------------------+--------------------------+-----------------------+--------------------------+--------------------------+---------------------------+----------------------+----------------------+-----------------------+--------------------+-------------------+-------------------+-------------------+--------------+----------------+---------------------------+------------------+--------------+-------------+-------------+-------------+-------------+------------+--------------------+--------------+------------

In [6]:
prev_raw.show()

+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|SK_ID_CURR|SK_ID_BUREAU|CREDIT_ACTIVE|CREDIT_CURRENCY|DAYS_CREDIT|CREDIT_DAY_OVERDUE|DAYS_CREDIT_ENDDATE|DAYS_ENDDATE_FACT|AMT_CREDIT_MAX_OVERDUE|CNT_CREDIT_PROLONG|AMT_CREDIT_SUM|AMT_CREDIT_SUM_DEBT|AMT_CREDIT_SUM_LIMIT|AMT_CREDIT_SUM_OVERDUE|    CREDIT_TYPE|DAYS_CREDIT_UPDATE|AMT_ANNUITY|
+----------+------------+-------------+---------------+-----------+------------------+-------------------+-----------------+----------------------+------------------+--------------+-------------------+--------------------+----------------------+---------------+------------------+-----------+
|    215354|     5714462|       Closed|     currency 1|       -497|                 0|             -153.0|           -153

<h2>Data prerperation</h2>

In [7]:
app_df = app_raw[['SK_ID_CURR','NAME_CONTRACT_TYPE','CODE_GENDER','FLAG_OWN_CAR','FLAG_OWN_REALTY','CNT_CHILDREN','AMT_INCOME_TOTAL','AMT_CREDIT','AMT_ANNUITY','AMT_GOODS_PRICE','NAME_TYPE_SUITE','NAME_INCOME_TYPE',
'NAME_EDUCATION_TYPE','NAME_FAMILY_STATUS', 'NAME_HOUSING_TYPE','DAYS_BIRTH','DAYS_EMPLOYED','DAYS_REGISTRATION','DAYS_ID_PUBLISH','OWN_CAR_AGE','OCCUPATION_TYPE','CNT_FAM_MEMBERS','WEEKDAY_APPR_PROCESS_START',
'HOUR_APPR_PROCESS_START','ORGANIZATION_TYPE','AMT_REQ_CREDIT_BUREAU_HOUR','AMT_REQ_CREDIT_BUREAU_DAY','AMT_REQ_CREDIT_BUREAU_WEEK','AMT_REQ_CREDIT_BUREAU_MON','AMT_REQ_CREDIT_BUREAU_QRT','AMT_REQ_CREDIT_BUREAU_YEAR']]
app_df.show()

+----------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+-----------------+---------------+-----------+---------------+---------------+--------------------------+-----------------------+--------------------+--------------------------+-------------------------+--------------------------+-------------------------+-------------------------+--------------------------+
|SK_ID_CURR|NAME_CONTRACT_TYPE|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|AMT_CREDIT|AMT_ANNUITY|AMT_GOODS_PRICE|NAME_TYPE_SUITE|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|DAYS_REGISTRATION|DAYS_ID_PUBLISH|OWN_CAR_AGE|OCCUPATION_TYPE|CNT_FAM_MEMBERS|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|   ORGANIZATION_TYPE|AMT_REQ_CREDIT

<h3>Find completness</h3>

In [15]:
def get_completeness(df_given):
    # Calculate completeness for each column
    completeness_data = []

    for column in df_given.columns:
        non_null_count = df_given.filter(col(column).isNotNull()).count()
        total_count = df_given.count()
        completeness = (non_null_count / total_count) * 100
        completeness_data.append((column, completeness))

    # Create a DataFrame to store the completeness results
    return spark.createDataFrame(completeness_data, ["Column", "Completeness"])


In [20]:
app_df_completness = get_completeness(app_df)
prev_raw_completness = get_completeness(prev_raw)

In [22]:
app_df_completness.show(50)
prev_raw_completness.show()

+--------------------+------------------+
|              Column|      Completeness|
+--------------------+------------------+
|          SK_ID_CURR|             100.0|
|  NAME_CONTRACT_TYPE|             100.0|
|         CODE_GENDER|             100.0|
|        FLAG_OWN_CAR|             100.0|
|     FLAG_OWN_REALTY|             100.0|
|        CNT_CHILDREN|             100.0|
|    AMT_INCOME_TOTAL|             100.0|
|          AMT_CREDIT|             100.0|
|         AMT_ANNUITY| 99.99609770057006|
|     AMT_GOODS_PRICE| 99.90959672987307|
|     NAME_TYPE_SUITE| 99.57985242804322|
|    NAME_INCOME_TYPE|             100.0|
| NAME_EDUCATION_TYPE|             100.0|
|  NAME_FAMILY_STATUS|             100.0|
|   NAME_HOUSING_TYPE|             100.0|
|          DAYS_BIRTH|             100.0|
|       DAYS_EMPLOYED|             100.0|
|   DAYS_REGISTRATION|             100.0|
|     DAYS_ID_PUBLISH|             100.0|
|         OWN_CAR_AGE|34.009189915157506|
|     OCCUPATION_TYPE| 68.65445463

<h3>Fill incomplete columns</h3>

In [28]:
fill_values = {"NAME_TYPE_SUITE" : "None",
                "OCCUPATION_TYPE" : "Unknown",
                'OWN_CAR_AGE' : 0,'CNT_FAM_MEMBERS' : 0,
                'AMT_ANNUITY' : 0,'AMT_GOODS_PRICE' : 0,
                'AMT_REQ_CREDIT_BUREAU_HOUR' : 0,
                'AMT_REQ_CREDIT_BUREAU_DAY' : 0,
                'AMT_REQ_CREDIT_BUREAU_WEEK' : 0,
                'AMT_REQ_CREDIT_BUREAU_MON' : 0,
                'AMT_REQ_CREDIT_BUREAU_QRT' : 0,
                'AMT_REQ_CREDIT_BUREAU_YEAR' : 0}
app_df_complete = app_df.fillna(fill_values)
completness_check = get_completeness(app_df_complete)

In [26]:
completness_check.show(50)

+--------------------+------------+
|              Column|Completeness|
+--------------------+------------+
|          SK_ID_CURR|       100.0|
|  NAME_CONTRACT_TYPE|       100.0|
|         CODE_GENDER|       100.0|
|        FLAG_OWN_CAR|       100.0|
|     FLAG_OWN_REALTY|       100.0|
|        CNT_CHILDREN|       100.0|
|    AMT_INCOME_TOTAL|       100.0|
|          AMT_CREDIT|       100.0|
|         AMT_ANNUITY|       100.0|
|     AMT_GOODS_PRICE|       100.0|
|     NAME_TYPE_SUITE|       100.0|
|    NAME_INCOME_TYPE|       100.0|
| NAME_EDUCATION_TYPE|       100.0|
|  NAME_FAMILY_STATUS|       100.0|
|   NAME_HOUSING_TYPE|       100.0|
|          DAYS_BIRTH|       100.0|
|       DAYS_EMPLOYED|       100.0|
|   DAYS_REGISTRATION|       100.0|
|     DAYS_ID_PUBLISH|       100.0|
|         OWN_CAR_AGE|       100.0|
|     OCCUPATION_TYPE|       100.0|
|     CNT_FAM_MEMBERS|       100.0|
|WEEKDAY_APPR_PROC...|       100.0|
|HOUR_APPR_PROCESS...|       100.0|
|   ORGANIZATION_TYPE|      

In [56]:
app_df = app_df_complete

<h3>Day values</h3>

In [57]:
columns_to_multiply = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']

for col_name in columns_to_multiply:
    app_df = app_df.withColumn(col_name, when(col(col_name) < 0, col(col_name) * -1).otherwise(col(col_name)))
app_df.show()

+----------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+-----------------+---------------+-----------+---------------+---------------+--------------------------+-----------------------+--------------------+--------------------------+-------------------------+--------------------------+-------------------------+-------------------------+--------------------------+
|SK_ID_CURR|NAME_CONTRACT_TYPE|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|AMT_CREDIT|AMT_ANNUITY|AMT_GOODS_PRICE|NAME_TYPE_SUITE|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|DAYS_REGISTRATION|DAYS_ID_PUBLISH|OWN_CAR_AGE|OCCUPATION_TYPE|CNT_FAM_MEMBERS|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START|   ORGANIZATION_TYPE|AMT_REQ_CREDIT

<h3>Renaming columns</h3>

In [37]:
app_df_renamed= app_df.withColumnRenamed('NAME_CONTRACT_TYPE', 'CONTRACT_TYPE') \
    .withColumnRenamed('CODE_GENDER', 'GENDER') \
    .withColumnRenamed('FLAG_OWN_CAR', 'OWN_CAR') \
    .withColumnRenamed('FLAG_OWN_REALTY', 'OWN_REALTY') \
    .withColumnRenamed('CNT_CHILDREN', 'NUMBER_CHILDREN') \
    .withColumnRenamed('AMT_INCOME_TOTAL', 'AMT_INCOME_TOTAL') \
    .withColumnRenamed('AMT_CREDIT', 'AMT_CREDIT') \
    .withColumnRenamed('AMT_ANNUITY', 'AMT_ANNUITY') \
    .withColumnRenamed('AMT_GOODS_PRICE', 'AMT_GOODS_PRICE') \
    .withColumnRenamed('NAME_TYPE_SUITE', 'REFERENCE_TYPE') \
    .withColumnRenamed('NAME_INCOME_TYPE', 'INCOME_TYPE') \
    .withColumnRenamed('NAME_EDUCATION_TYPE', 'EDUCATION_TYPE') \
    .withColumnRenamed('NAME_FAMILY_STATUS', 'FAMILY_STATUS') \
    .withColumnRenamed('NAME_HOUSING_TYPE', 'HOUSING_TYPE') \
    .withColumnRenamed('REGION_POPULATION_RELATIVE', 'REGION_POPULATION_RELATIVE') \
    .withColumnRenamed('DAYS_BIRTH', 'DAYS_BIRTH') \
    .withColumnRenamed('DAYS_EMPLOYED', 'DAYS_EMPLOYED') \
    .withColumnRenamed('DAYS_REGISTRATION', 'DAYS_REGISTRATION') \
    .withColumnRenamed('DAYS_ID_PUBLISH', 'DAYS_ID_PUBLISH') \
    .withColumnRenamed('OWN_CAR_AGE', 'CAR_AGE') \
    .withColumnRenamed('OCCUPATION_TYPE', 'OCCUPATION_TYPE') \
    .withColumnRenamed('CNT_FAM_MEMBERS', 'NUMBER_OF_FAM_MEMBERS') \
    .withColumnRenamed('WEEKDAY_APPR_PROCESS_START', 'WEEKDAY_PROCESS_START') \
    .withColumnRenamed('HOUR_APPR_PROCESS_START', 'HOUR_PROCESS_START') \
    .withColumnRenamed('ORGANIZATION_TYPE', 'ORGANIZATION_TYPE') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_HOUR', 'CREDIT_INQUIRY_HOUR_PRIOR') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_DAY', 'CREDIT_INQUIRY_DAY_PRIOR') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_WEEK', 'CREDIT_INQUIRY_WEEK_PRIOR') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_MON', 'CREDIT_INQUIRY_MONTH_PRIOR') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_YEAR', 'CREDIT_INQUIRY_YEAR_PRIOR') \
    .withColumnRenamed('AMT_REQ_CREDIT_BUREAU_QRT', 'CREDIT_INQUIRY_QTR_PRIOR') 

In [38]:
app_df_renamed.show()

+----------+---------------+------+-------+----------+---------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+----------+-------------+-----------------+---------------+-------+---------------+---------------------+---------------------+------------------+--------------------+-------------------------+------------------------+-------------------------+--------------------------+------------------------+-------------------------+
|SK_ID_CURR|  CONTRACT_TYPE|GENDER|OWN_CAR|OWN_REALTY|NUMBER_CHILDREN|AMT_INCOME_TOTAL|AMT_CREDIT|AMT_ANNUITY|AMT_GOODS_PRICE| REFERENCE_TYPE|         INCOME_TYPE|      EDUCATION_TYPE|       FAMILY_STATUS|     HOUSING_TYPE|DAYS_BIRTH|DAYS_EMPLOYED|DAYS_REGISTRATION|DAYS_ID_PUBLISH|CAR_AGE|OCCUPATION_TYPE|NUMBER_OF_FAM_MEMBERS|WEEKDAY_PROCESS_START|HOUR_PROCESS_START|   ORGANIZATION_TYPE|CREDIT_INQUIRY_HOUR_PRIOR|CREDIT_INQUIRY_DAY_PRIOR|CREDIT_INQUIR

<h3>Make year values</h3>

In [58]:
columns_to_multiply = ['DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION', 'DAYS_ID_PUBLISH']

for col_name in columns_to_multiply:
    app_df = app_df.withColumn(col_name, col(col_name)/365)
app_df = app_df.withColumnRenamed('DAYS_BIRTH', 'Age') \
    .withColumnRenamed('DAYS_EMPLOYED', 'Years_EMPLOYED') \
    .withColumnRenamed('DAYS_REGISTRATION', 'Years_REGISTRATION') \
    .withColumnRenamed('DAYS_ID_PUBLISH', 'Years_ID_PUBLISH')

In [59]:
app_df.show()

+----------+------------------+-----------+------------+---------------+------------+----------------+----------+-----------+---------------+---------------+--------------------+--------------------+--------------------+-----------------+------------------+------------------+------------------+------------------+-----------+---------------+---------------+--------------------------+-----------------------+--------------------+--------------------------+-------------------------+--------------------------+-------------------------+-------------------------+--------------------------+
|SK_ID_CURR|NAME_CONTRACT_TYPE|CODE_GENDER|FLAG_OWN_CAR|FLAG_OWN_REALTY|CNT_CHILDREN|AMT_INCOME_TOTAL|AMT_CREDIT|AMT_ANNUITY|AMT_GOODS_PRICE|NAME_TYPE_SUITE|    NAME_INCOME_TYPE| NAME_EDUCATION_TYPE|  NAME_FAMILY_STATUS|NAME_HOUSING_TYPE|               Age|    Years_EMPLOYED|Years_REGISTRATION|  Years_ID_PUBLISH|OWN_CAR_AGE|OCCUPATION_TYPE|CNT_FAM_MEMBERS|WEEKDAY_APPR_PROCESS_START|HOUR_APPR_PROCESS_START| 